<a href="https://colab.research.google.com/github/Talha1818/Vector-Database-Management-System-Word-Emdedding-ChromaDB/blob/master/Vector_Database_Management_System_Word_Emdedding_ChromaDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.2 MB/s eta 0:00:00


In [ ]:
from transformers import logging
logging.set_verbosity_error()

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch

def get_bert_embedding(text):
    # Load the latest BERT model and tokenizer
    model_name = 'bert-base-uncased'
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    # Set the device to 'cuda' if available, otherwise use 'cpu'
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Tokenize the input text and convert it to input tensors
    max_length = tokenizer.model_max_length - 2  # Account for special tokens
    input_tokens = tokenizer.encode_plus(text, add_special_tokens=True, truncation=True, max_length=max_length, return_tensors='pt')
    input_tokens = {key: tensor.to(device) for key, tensor in input_tokens.items()}

    # Generate the BERT embeddings
    with torch.no_grad():
        model.eval()
        output = model(**input_tokens)

    # Access the embedding for the first token ([CLS] token)
    embedding = output.last_hidden_state[:, 0, :]

    # Convert the embedding to a numpy array
    embedding = embedding.cpu().numpy()

    return embedding[0]

In [ ]:
input_text = "Sample document text goes here"
embedding = get_bert_embedding(input_text)
embedding.shape

(768,)

In [ ]:
import re

def remove_special_chars(text):
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    cleaned_text = re.sub(r'[\n\r]', ' ', cleaned_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text
# Example usage
text = "Hello! \n\r How are/you doing today?        #Excited".replace("/", " ")
cleaned_text = remove_special_chars(text)
print(cleaned_text)

Hello How are you doing today Excited


In [ ]:
import os
import requests
import json
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt
import glob
import re
import nltk
from nltk.corpus import stopwords
import traceback
nltk.download('stopwords')
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")


class credentials:
    def __init__(self, filename:str):
        self.filename = filename
    def get_credentials(self):
        with open(self.filename, 'r') as openfile:
            credentials = json.load(openfile)
        server = credentials['server']
        database = credentials['database']
        email = credentials['email']
        password = credentials['password']
        print("✅ Credentials Loaded.")
        return server, database, email, password

class Authorization:
    def __init__(self, server, email, password):
        self.server = server
        self.email = email
        self.password = password
        self.payload={'password': password,'email': email}
        self.response = requests.request("POST",  server+"authenticate/", data=self.payload)
    def check_authentication(self):
        headers = {"Authorization": ""}
        if self.response.status_code == 200:
            # print(json.loads(self.response.text)['access_token'])
            # headers = {"Authorization": "Bearer "+json.loads(self.response.text)[0]['access_token']}
            headers = {"Authorization": "Bearer "+json.loads(self.response.text)['data']['access_token']}
            print("✅ User Authenticated.")
            return headers
        else:
            print("❌ User Not Authenticated.")
            return headers

class Database:
    def __init__(self, server, database, headers):
        self.server = server
        self.database = database
        self.headers = headers
        self.response = requests.get(server+database, headers=self.headers)
    def get_json(self):
        if self.response.status_code == 200:
            data = self.response.json()
            print("✅ database loaded.")
            return data
        else:
            print("❌ Request failed with status code", self.response.status_code)
            return {
                "server":self.server,
                "database":self.database,
                "headers":self.headers,
                "status_code":self.response.status_cod
            }
def GetDatasetDatFrame(credential_filename):
  cred = credentials(credential_filename)
  server, _, email, password= cred.get_credentials()
  auth = Authorization(server, email, password)
  headers = auth.check_authentication()

  url = server+"job/?page=1&ready_for_dataset=1"

  packages_list = []
  packages_name_list = []
  document_type = []
  document_text = []

  def checking_docs(url):
    response = requests.get(url, headers=headers)
    if response.json()['data']['next'] != None:
      next = response.json()['data']['next']
      for i in tqdm(response.json()['data']['results'], desc="Processing"):
        # print(i['file_path'], "==>", i['package_address'])
        # packages_name_list.append(i['package_address'])
        # packages_list.append(i['file_path'].split("/")[-1])
        package_id = server+f"job/{i['id']}/"
        response1 = requests.get(package_id, headers=headers)
        for j in tqdm(response1.json()['data']['job']):
          packages_name_list.append(i['package_address'])
          packages_list.append(i['file_path'].split("/")[-1])
          document_text.append(j["document_text"])
          document_type.append(j["document_type"])
    else:
      return "No More Next."
    return checking_docs(next)

  checking_docs(url)
  df2 = pd.DataFrame()
  df2['Packages'] = packages_list
  df2['Packages_Name'] = packages_name_list
  df2['text'] = document_text
  df2['token'] = document_type

  return df2

class CleanText:
  def __init__(self) -> None:
     pass
  # Define cleaning function
  def clean_text(self, text):
    text = text.replace("/", " ")
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    cleaned_text = re.sub(r'[\n\r]', ' ', cleaned_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

class ConcatenateAllDatasetUpdate(CleanText):
  def __init__(self, df):
    super().__init__()  # Calling the parent class constructor
    self.df = df
    self.df = self.df[~(self.df['text']=='')]
    self.df = self.df[~(self.df['token']=='')]
    self.df = self.df[~(self.df['token']==' ')]
    self.df = self.df[self.df['text'].notna()]
    self.df = self.df[self.df['token'].notna()]
    print("Total Dataset Shape: ", self.df.shape)

  def get_clean_dataset(self):
    assert len(self.df)!=0, "DataFrame is Empty plz run again to add more files."
    self.df['clean_text'] = self.df['text'].apply(super().clean_text)
    # return self.df[['clean_text', 'token']]
    return self.df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df_ = GetDatasetDatFrame("/content/credentials-Dataset.json")
cls = ConcatenateAllDatasetUpdate(df_)
df = cls.get_clean_dataset()

✅ Credentials Loaded.
✅ User Authenticated.


Processing:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/113 [00:00<?, ?it/s]

  0%|          | 0/89 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/118 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/102 [00:00<?, ?it/s]

Total Dataset Shape:  (1037, 4)


In [ ]:
df.head()

,Packages,Packages_Name,text,token,clean_text
0,test_unidentified_987944.pdf,Default Package,"City Abstract, LLC\r\n1050 Kings Highway North...",609,City Abstract LLC 1050 Kings Highway North Sui...
1,test_unidentified_987944.pdf,Default Package,Acknowledgement\nWe/I have carefully reviewed ...,300,Acknowledgement We I have carefully reviewed t...
2,test_unidentified_987944.pdf,Default Package,STATE OF\nCOUNTY OF\n1.\n2.\n3.\nMichael Josep...,290,STATE OF COUNTY OF 1 2 3 Michael Joseph Fee sa...
3,test_unidentified_987944.pdf,Default Package,7.\n8.\n9.\nExceptions and Additions. The foll...,290,7 8 9 Exceptions and Additions The following i...
4,test_unidentified_987944.pdf,Default Package,Closing Disclosure\nClosing Information\nDate ...,301,Closing Disclosure Closing Information Date Is...


In [ ]:
df.shape

(1037, 5)

In [ ]:
# df = df.iloc[:100]

In [ ]:
embeding_list = []
for text in tqdm(df['clean_text']):
  embeding_list.append(get_bert_embedding(text))

  0%|          | 0/1037 [00:00<?, ?it/s]

In [ ]:
df['word_embedding'] = embeding_list

In [ ]:
df.head()

,Packages,Packages_Name,text,token,clean_text,word_embedding
0,test_unidentified_987944.pdf,Default Package,"City Abstract, LLC\r\n1050 Kings Highway North...",609,City Abstract LLC 1050 Kings Highway North Sui...,"[-0.51558137, 0.28453466, 0.1843392, 0.0125034..."
1,test_unidentified_987944.pdf,Default Package,Acknowledgement\nWe/I have carefully reviewed ...,300,Acknowledgement We I have carefully reviewed t...,"[-0.7214845, 0.21466307, 0.23127192, 0.0755820..."
2,test_unidentified_987944.pdf,Default Package,STATE OF\nCOUNTY OF\n1.\n2.\n3.\nMichael Josep...,290,STATE OF COUNTY OF 1 2 3 Michael Joseph Fee sa...,"[-0.14801769, 0.08115877, 0.43437764, -0.00477..."
3,test_unidentified_987944.pdf,Default Package,7.\n8.\n9.\nExceptions and Additions. The foll...,290,7 8 9 Exceptions and Additions The following i...,"[-0.680785, 0.19464856, 0.18208224, -0.1188453..."
4,test_unidentified_987944.pdf,Default Package,Closing Disclosure\nClosing Information\nDate ...,301,Closing Disclosure Closing Information Date Is...,"[-0.5463826, 0.16736032, 0.3614848, -0.0608652..."


In [ ]:
df.to_csv("vector-database.csv", index=False)

# **Upload/Download Database to an AWS S3 BUCKET**

In [ ]:
!pip install --upgrade transformers
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.2 MB/s eta 0:00:00


In [ ]:
import os
import requests
import json
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt
import glob
import re
import nltk
from nltk.corpus import stopwords
import traceback
nltk.download('stopwords')
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
aws_keys = pd.read_csv("talha_accessKeys.csv")

import boto3
import botocore
from botocore.exceptions import ClientError
import os

BUCKET_NAME = 'documentapplication' # replace with your bucket name

# enter authentication credentials
s3 = boto3.resource('s3', aws_access_key_id = aws_keys['Access key ID'].values[0],
                          aws_secret_access_key = aws_keys['Secret access key'].values[0])

# Check if the bucket already exists
try:
    s3.Bucket(BUCKET_NAME).load()
    print('Bucket already exists')
except ClientError as e:
    # If the bucket does not exist, create it with the specified region
    if e.response['Error']['Code'] == "404":
        try:
            my_region = boto3.session.Session().region_name # set the region of the instance
            s3.create_bucket(Bucket=BUCKET_NAME, CreateBucketConfiguration={'LocationConstraint': my_region})
            print('Bucket created successfully')
        except Exception as e:
            print('S3 error: ', e)
    else:
        print('S3 error: ', e)

Bucket already exists


In [ ]:
def upload_file_to_s3(bucket_name=None, file_name="vector-database.csv", key_prefix=""):
    key_prefix_with_file_name = os.path.join(key_prefix, file_name)
    s3.Bucket(bucket_name).upload_file(file_name, key_prefix_with_file_name)
    return f"s3://{bucket_name}/{key_prefix_with_file_name}"

In [ ]:
def download_file_from_s3(bucket_name=None, file_name="vector-database.csv", key_prefix=""):
    key_prefix_with_file_name = os.path.join(key_prefix, file_name)
    s3.Bucket(bucket_name).download_file(key_prefix_with_file_name, file_name)
    return f"s3://{bucket_name}/{key_prefix_with_file_name}"

In [ ]:
# upload to s3
file_name = "vector-database.csv"
print(
    f"uploading `{file_name}` archive to s3"
)

prefix = "Vector-Based-Database"
model_uri = upload_file_to_s3(bucket_name=BUCKET_NAME,
                              file_name = file_name,
                              key_prefix=prefix,
                              )

print(f"Successfully uploaded to {model_uri}")

uploading `vector-database.csv` archive to s3
Successfully uploaded to s3://documentapplication/Vector-Based-Database/vector-database.csv


In [ ]:
# download from s3
file_name = "vector-database.csv"
print(
    f"downloading `{file_name}` archive from s3"
)

prefix = "Vector-Based-Database"
model_uri = download_file_from_s3(bucket_name=BUCKET_NAME,
                              file_name = file_name,
                              key_prefix=prefix,
                              )

print(f"Successfully downloaded the {model_uri}")

downloading `vector-database.csv` archive from s3
Successfully downloaded the s3://documentapplication/Vector-Based-Database/vector-database.csv


In [ ]:
df = pd.read_csv(file_name)
df.head()

,Packages,Packages_Name,text,token,clean_text,word_embedding
0,test_unidentified_987944.pdf,Default Package,"City Abstract, LLC\r\n1050 Kings Highway North...",609,City Abstract LLC 1050 Kings Highway North Sui...,[-5.15581369e-01 2.84534663e-01 1.84339195e-...
1,test_unidentified_987944.pdf,Default Package,Acknowledgement\nWe/I have carefully reviewed ...,300,Acknowledgement We I have carefully reviewed t...,[-7.21484482e-01 2.14663073e-01 2.31271923e-...
2,test_unidentified_987944.pdf,Default Package,STATE OF\nCOUNTY OF\n1.\n2.\n3.\nMichael Josep...,290,STATE OF COUNTY OF 1 2 3 Michael Joseph Fee sa...,[-1.48017690e-01 8.11587721e-02 4.34377640e-...
3,test_unidentified_987944.pdf,Default Package,7.\n8.\n9.\nExceptions and Additions. The foll...,290,7 8 9 Exceptions and Additions The following i...,[-6.80785000e-01 1.94648564e-01 1.82082236e-...
4,test_unidentified_987944.pdf,Default Package,Closing Disclosure\nClosing Information\nDate ...,301,Closing Disclosure Closing Information Date Is...,[-5.46382606e-01 1.67360321e-01 3.61484796e-...


In [ ]:
################
############# WHEN WE EXCESS DATA THROUGH S3 THEN WE NEED TO RUN
############# THIS BLOCK OF CODE, ACTUALLY WHEN WE STORE DATA ON S3 THAT STORE THE WORD_EMBEDDING
############# FEATURE AS A STRING SO WE NEED TO CONVERT INTO NUMPY ARRAY FORMAT
##############


# Define a function to convert the string to array
def convert_to_array(embedding_str):
    return np.fromstring(embedding_str[1:-1], sep=' ')

# Apply the function to the 'word_embedding' column
df['word_embedding'] = df['word_embedding'].apply(lambda x: convert_to_array(x))

In [ ]:
def calculate_cosine_similarity(testing_text, dataframe, top=5):

    embedding = get_bert_embedding(testing_text)
    # Reshape the embedding to match the shape of the 'word_embedding' column
    embedding = embedding.reshape(1, -1)

    # Calculate the cosine similarity between the embedding and 'word_embedding' column
    similarity_scores = cosine_similarity(embedding, np.vstack(dataframe['word_embedding']))

    # Add the similarity scores as a new column in the DataFrame
    dataframe['similarity'] = similarity_scores[0]

    # Sort the DataFrame by similarity score in descending order and get the top 5 rows
    top_similar_rows = dataframe.sort_values(by='similarity', ascending=False).head(top)

    # Reset the index of the DataFrame
    top_similar_rows.reset_index(drop=True, inplace=True)

    # Remove the similarity column from the DataFrame
    #top_similar_rows.drop('similarity', axis=1, inplace=True)

    return top_similar_rows

In [ ]:
testing_text = df['clean_text'][90]
testing_text

'Borrower Name Geralda Manushi Notice to the Home Loan Applicant In connection with your application for a home loan the lender must disclose to you the score that a consumer reporting agency distributed to users and the lender used in connection with your home loan and the key factors affecting your credit scores The credit score is a computer generated summary calculated at the time of the request and based on information that a consumer reporting agency or lender has on file The scores are based on data about your credit history and payment patterns Credit scores are important because they are used to assist the lender in determining whether you will obtain a loan They may also be used to determine what interest rate you may be offered on the mortgage Credit scores can change over time depending on your conduct how your credit history and payment patterns change and how credit scoring technologies change Because the score is based on information in your credit history it is very imp

In [ ]:
# testing_text = "What does mean by closing documents."

In [ ]:
top = 3
similar_rows = calculate_cosine_similarity(testing_text, df, top)

# Print the resulting DataFrame
similar_rows

,Packages,Packages_Name,text,token,clean_text,word_embedding,similarity
0,CAAC-510699-NJ.pdf,Default Package,Borrower Name: Geralda Manushi\nNotice to the ...,328,Borrower Name Geralda Manushi Notice to the Ho...,"[-0.687112212, -0.0173120275, -0.23813121, 0.2...",1.000000
1,CAAC-510703-NJ.pdf,Default Package,Notice to the Home Loan Applicant\nIn connecti...,328,Notice to the Home Loan Applicant In connectio...,"[-0.652878165, 0.294668257, -0.155769467, 0.22...",0.927896
2,CAAC-510713-PA.pdf,Default Package,Borrower: Margretta Long\nProperty Address:\n4...,351,Borrower Margretta Long Property Address 4512 ...,"[-0.695419073, 0.307772666, -0.0889908075, 0.3...",0.922285


In [ ]:
context_list = [i for i in similar_rows['clean_text'].values.reshape(-1,1).tolist()]

In [ ]:
context_doc = ""
for i in context_list:
  for j in i:
    context_doc = context_doc + " "+ j

In [ ]:
context_doc

' Borrower Name Geralda Manushi Notice to the Home Loan Applicant In connection with your application for a home loan the lender must disclose to you the score that a consumer reporting agency distributed to users and the lender used in connection with your home loan and the key factors affecting your credit scores The credit score is a computer generated summary calculated at the time of the request and based on information that a consumer reporting agency or lender has on file The scores are based on data about your credit history and payment patterns Credit scores are important because they are used to assist the lender in determining whether you will obtain a loan They may also be used to determine what interest rate you may be offered on the mortgage Credit scores can change over time depending on your conduct how your credit history and payment patterns change and how credit scoring technologies change Because the score is based on information in your credit history it is very im

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering",  "Rocketknight1/distilbert-base-uncased-finetuned-squad", framework="tf")

In [ ]:
context = context_doc
query = "What is the borrower name?"
question_answerer(context=context, question=query)

{'score': 0.8302730321884155,
 'start': 15,
 'end': 30,
 'answer': 'Geralda Manushi'}

# **ChromaDB**

* Chroma is the open-source embedding database. Chroma makes it easy to build LLM apps by making knowledge, facts, and skills pluggable for LLMs.

In [ ]:
!pip install chromadb -q
!pip install sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.7/966.7 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s

In [ ]:
import chromadb

client = chromadb.Client()
collection = client.create_collection("yt_demo")

In [ ]:
collection.add(
    documents=["This is a document about cat", "This is a document about car"],
    metadatas=[{"category": "animal"}, {"category": "vehicle"}],
    ids=["id1", "id2"]
)


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 62.9MiB/s]


In [ ]:
results = collection.query(
    query_texts=["vehicle"],
    n_results=1
)
print(results)

{'ids': [['id2']], 'embeddings': None, 'documents': [['This is a document about car']], 'metadatas': [[{'category': 'vehicle'}]], 'distances': [[0.8069301247596741]]}


In [ ]:
from chromadb.config import Settings
client = chromadb.Client(Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory="doc_directory" # Optional, defaults to .chromadb/ in the current directory
))

In [ ]:
df = df.reset_index()
df.head(2)

,index,Packages,Packages_Name,text,token,clean_text,word_embedding,similarity
0,0,test_unidentified_987944.pdf,Default Package,"City Abstract, LLC\r\n1050 Kings Highway North...",609,City Abstract LLC 1050 Kings Highway North Sui...,"[-0.515581369, 0.284534663, 0.184339195, 0.012...",0.748090
1,1,test_unidentified_987944.pdf,Default Package,Acknowledgement\nWe/I have carefully reviewed ...,300,Acknowledgement We I have carefully reviewed t...,"[-0.721484482, 0.214663073, 0.231271923, 0.075...",0.724376


In [ ]:
documents = []
embeddings = []
metadatas = []
ids = []

for index, pkg_name, text, emb in tqdm(zip(df['index'],df['Packages_Name'],df['clean_text'], df['word_embedding'])):
    documents.append(text)
    embeddings.append(emb.tolist())
    metadatas.append({'source': pkg_name})
    ids.append(str(index + 1))

0it [00:00, ?it/s]

In [ ]:
metadatas[-1]

{'source': 'Default Package'}

In [ ]:
# df['word_embedding'][0].tolist()

In [ ]:
def add_collection(random_choice,  UseOwnembedding=False):
  document_collection_emb = client.create_collection(f"document_collection_emb{random_choice}")
  if UseOwnembedding:
    document_collection_emb.add(
    documents=documents,
    embeddings=embeddings,
    metadatas=metadatas,
    ids=ids)
  else:
    document_collection_emb.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids)
  return document_collection_emb

## **Use Own Embedding (BERT)**

In [ ]:
import random

choices = list(range(100000))  # Create a list of numbers from 0 to 99999
random_choice = random.choices(choices)[0]

# USE OWN EMBEDDING (BERT)
document_collection_emb = add_collection(random_choice,UseOwnembedding=True)

In [ ]:
query = "What is the Closing Documents?"
# cls = CleanText()

input_em = get_bert_embedding(query).tolist()

results = document_collection_emb.query(
    query_embeddings=[input_em],
    n_results=3
)
results

{'ids': [['218', '944', '830']],
 'embeddings': None,
 'documents': [['Confirm Receipt Closing Disclosure Miriam OConnor har Miriam Stacy 2 5 2050 ا تو',
   'Confirm Receipt 17 Laut Boson Martin Basso Keußen 2 27 20 Kalynn Basso 2272020 Date Fax 3',
   'Confirm Receipt Closing Didosure Estate of Dorothy Chato Chatot By Roger Chatot Executor Chebt 8 19 21 Date Page 2 I 1 1']],
 'metadatas': [[{'source': 'Default Package'},
   {'source': 'Default Package'},
   {'source': 'Default Package'}]],
 'distances': [[45.308326721191406, 45.33436584472656, 48.00537109375]]}

## **Use by-default Embedding (Sentence Transformers)**

In [ ]:
import random

choices = list(range(100000))  # Create a list of numbers from 0 to 99999
random_choice = random.choices(choices)[0]

document_collection_emb = add_collection(random_choice, UseOwnembedding=False)

# USE BY-DEAFULT EMBEDDING (Sentence-Transformer)
results = document_collection_emb.query(
    query_texts=query,
    n_results=3
)
results

{'ids': [['46', '257', '265']],
 'embeddings': None,
 'documents': [['Signatures Corrections Closing Disclosure Power of Attorney FHA or VA Occupancy BORROWERS MUST SIGN ALL DOCUMENTS EXACTLY AS HIS OR HER NAME APPEARS ON THE BLANK LINE PROVIDED FOR HIS OR HER SIGNATURES Where a witness is required you must have the document witnessed If an acknowledgment is provided a person authorized to take acknowledgments in the state of closing must execute the acknowledgment The acknowledgment date must be no earlier than the date of the document being acknowledged Any correction to loan documents must be approved in writing by us in advance NO WHITEOUT PERMITTED Approved deletion should be made by marking a single line through the language being deleted All additions and deletions must be initialed by all borrowers The Borrowers and Sellers Closing Disclosures must clearly indicate the recipient of each item shown on the disclo sures Both Borrowers and Sellers must sign the Closing Disclosure i